In [4]:
import pandas as pd
import numpy as np
import datetime

# --- 1. DATA GENERATION ---
def generate_supply_chain_data():
    # sales.csv — daily item demand
    dates = pd.date_range(start='2025-10-01', periods=90)
    sales_df = pd.DataFrame({
        'date': dates,
        'sku': 'SKU_WIDGET_A',
        'qty_sold': np.random.poisson(lam=12, size=90)
    })
    sales_df.to_csv('sales.csv', index=False)

    # inventory.csv — starting stock
    pd.DataFrame({
        'sku': ['SKU_WIDGET_A'],
        'opening_stock': [150]
    }).to_csv('inventory.csv', index=False)

    # params.csv — policy & cost inputs
    pd.DataFrame({
        'sku': ['SKU_WIDGET_A'],
        'unit_cost': [15.0],
        'holding_cost_per_day': [0.20],
        'stockout_cost': [50.0],
        'lead_time_days': [5],
        'min_order_qty': [100],
        'service_level': [1.96] # Targets a 97.5% fill rate
    }).to_csv('params.csv', index=False)

# --- 2. AGENT DEFINITION ---
class ReplenishmentAgent:
    """
    Agent Role: Autonomous inventory manager using EWMA forecasting
    and safety stock guardrails to prevent stockouts. [cite: 7, 10, 22]
    """
    def __init__(self, params):
        self.params = params

    def analyze_and_decide(self, sku, current_stock, open_orders, history):
        p = self.params[self.params['sku'] == sku].iloc[0]

        # Build EWMA forecast (Last 10 days)
        forecast_daily = history['qty_sold'].ewm(span=10).mean().iloc[-1]
        std_dev = history['qty_sold'].tail(14).std()

        # Compute Safety Stock based on service level
        safety_stock = p['service_level'] * std_dev * np.sqrt(p['lead_time_days'])

        # Calculate Reorder Point (ROP)
        reorder_point = (forecast_daily * p['lead_time_days']) + safety_stock

        # Total inventory position
        inv_position = current_stock + sum(order['qty'] for order in open_orders)

        # Decision logic
        if inv_position < reorder_point:
            order_qty = max(p['min_order_qty'], int(reorder_point - inv_position + forecast_daily))
            rationale = f"REORDER: Position {inv_position:.1f} < ROP {reorder_point:.1f}. Forecast: {forecast_daily:.1f}/day."
            return order_qty, rationale

        return 0, "HOLD: Stock levels sufficient."

# --- 3. SIMULATION RUNTIME ---
generate_supply_chain_data()
sales_df = pd.read_csv('sales.csv', parse_dates=['date'])
params_df = pd.read_csv('params.csv')
inv_df = pd.read_csv('inventory.csv')

agent = ReplenishmentAgent(params_df)
stock_on_hand = inv_df.iloc[0]['opening_stock']
pending_pos = [] # Orders currently in transit
metrics = {'demanded': 0, 'sold': 0, 'stockout_costs': 0}

print(f"{'DATE':<12} | {'STOCK':<6} | {'ORDER':<6} | {'AGENT RATIONALE'}")
print("-" * 90)

# Run simulation (Days 30 to 90)
for i in range(30, 90):
    day_data = sales_df.iloc[i]
    history = sales_df.iloc[:i]
    demand = day_data['qty_sold']
    metrics['demanded'] += demand

    # Process arriving POs (Lead time latency)
    for po in pending_pos[:]:
        if po['eta'] <= day_data['date']:
            stock_on_hand += po['qty']
            pending_pos.remove(po)

    # Agent Decision Loop [cite: 9, 21]
    order_qty, rationale = agent.analyze_and_decide('SKU_WIDGET_A', stock_on_hand, pending_pos, history)

    if order_qty > 0:
        eta = day_data['date'] + pd.Timedelta(days=int(params_df.iloc[0]['lead_time_days']))
        pending_pos.append({'eta': eta, 'qty': order_qty})

    # Fulfill daily demand
    sale = min(stock_on_hand, demand)
    metrics['sold'] += sale
    stockout = max(0, demand - stock_on_hand)
    metrics['stockout_costs'] += stockout * params_df.iloc[0]['stockout_cost']
    stock_on_hand -= sale

    # Log daily activity
    if i < 45: # Log first 15 days for review
        print(f"{str(day_data['date'].date()):<12} | {stock_on_hand:<6} | {order_qty:<6} | {rationale}")

# --- 4. EVALUATION ---
fill_rate = (metrics['sold'] / metrics['demanded']) * 100
print("-" * 90)
print(f"SIMULATION RESULTS [cite: 21, 22]")
print(f"Total Fill Rate: {fill_rate:.2f}%")
print(f"Total Stockout Cost Penalty: ${metrics['stockout_costs']:,.2f}")

DATE         | STOCK  | ORDER  | AGENT RATIONALE
------------------------------------------------------------------------------------------
2025-10-31   | 140    | 0      | HOLD: Stock levels sufficient.
2025-11-01   | 127    | 0      | HOLD: Stock levels sufficient.
2025-11-02   | 108    | 0      | HOLD: Stock levels sufficient.
2025-11-03   | 92     | 0      | HOLD: Stock levels sufficient.
2025-11-04   | 79     | 0      | HOLD: Stock levels sufficient.
2025-11-05   | 70     | 100    | REORDER: Position 79.0 < ROP 81.9. Forecast: 13.1/day.
2025-11-06   | 61     | 0      | HOLD: Stock levels sufficient.
2025-11-07   | 42     | 0      | HOLD: Stock levels sufficient.
2025-11-08   | 29     | 0      | HOLD: Stock levels sufficient.
2025-11-09   | 20     | 0      | HOLD: Stock levels sufficient.
2025-11-10   | 108    | 0      | HOLD: Stock levels sufficient.
2025-11-11   | 96     | 0      | HOLD: Stock levels sufficient.
2025-11-12   | 82     | 0      | HOLD: Stock levels sufficient.
2025